## Customer Churn Pipelines

Import library yang dibutuhkan

In [1]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, transform_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

2024-11-06 21:03:13.190488: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 21:03:13.243038: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Mendefinisikan global variable yang digunakan pada pipeline.

In [2]:
PIPELINE_NAME = 'churn-pipeline'
SCHEMA_PIPELINE_NAME = 'churn-tfdv-schema'

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = 'data'

Mendefinisikan `Interactive Context` untuk melakukan running compenent pipeline secara interaktif.

In [3]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

Melakukan import dataset dari local storage ke dalam pipeline, kedua dataset ini sudah terpisah antara data training dan data testing sehingga tidak perlu dilakukan splitting data.

In [4]:
input = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='*training*'),
    example_gen_pb2.Input.Split(name='eval', pattern='*testing*')
])
example_gen = CsvExampleGen(input_base=DATA_ROOT, input_config=input)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Selanjutnya dengan `StatisticGen` akan melakukan analisis statistik dari dataset.

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan hasil analisis statistik dari dataset dengan mengambil output dari `StatisticGen`.

In [6]:
context.show(statistics_gen.outputs['statistics'])

Berdasarkan hasil statistik di atas, dapat diketahui ukuran dari dataset, jumlah missing value, dan distribusi dari data. Masing-masing jenis data sudah dipisahkan berdasarkan tipenya yaitu numerical dan categorical. Dari sini dapat diketahui bahwa tidak terdapat nilai yang janggal pada dataset. Untuk memastikan hal itu, nantinya dapat digunakan component `ExampleValidator` untuk melakukan pengecekan anomaly pada data.

Dengan `SchemaGen` akan melakukan analisis skema dari dataset yang digunakan untuk mengetahui tipe data dari masing-masing kolom.

In [7]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'Churn',INT,required,,-
'Contract Length',STRING,required,,'Contract Length'
'CustomerID',INT,required,,-
'Gender',STRING,required,,'Gender'
'Last Interaction',INT,required,,-
'Payment Delay',INT,required,,-
'Subscription Type',STRING,required,,'Subscription Type'
'Support Calls',INT,required,,-


,Values
Domain,
'Contract Length',"'Annual', 'Monthly', 'Quarterly'"
'Gender',"'Female', 'Male'"
'Subscription Type',"'Basic', 'Premium', 'Standard'"


Dari output di atas, dapat diketahui tipe data dari masing-masing kolom pada dataset. Dari sini dapat diketahui bahwa terdapat 19 kolom pada dataset, dimana 8 kolom bertipe data `INT`, 3 kolom bertipe data `STRING`, dan 1 kolom bertipe data `FLOAT`.

Selanjutnya, dengan `ExampleValidator` akan melakukan validasi data untuk mengecek apakah terdapat data yang janggal pada dataset.

In [9]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
context.show(example_validator.outputs['anomalies'])

Dari output di atas, dapat diketahui bahwa tidak terdapat data yang janggal pada dataset. Hal ini dapat dilihat dari nilai `No anomalies found`.

Langkah selanjutnya adalah melakukan preprocessing data dengan `Transform` untuk melakukan transformasi data yang dibutuhkan. Pada kasus ini, akan dilakukan transformasi data dengan mengubah data kategorikal menjadi data numerik dengann menggunakan one hot encoding. Selain itu, akan dilakukan scaling data dengan menggunakan `MinMaxScaler` dengan `tft.scale_to_0_1` untuk melakukan scaling data ke dalam rentang 0 hingga 1.

In [11]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(os.path.join('modules', 'transform.py')),
)

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
copying tuner.py -> build/lib


/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmpa9cwrg_8
running install
running install_lib
copying build/lib/tuner.py -> /tmp/tmpa9cwrg_8
copying build/lib/trainer.py -> /tmp/tmpa9cwrg_8
copying build/lib/transform.py -> /tmp/tmpa9cwrg_8
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpa9cwrg_8/tfx_user_code_Transform-0.0+352f0552656f62183b3c2213824430061e98d4ea3a9b5075f5af3274b4849e9e-py3.10.egg-info
running install_scripts
creating /tmp/tmpa9cwrg_8/tfx_user_code_Transform-0.0+352f0552656f62183b3c2213824430061e98d4ea3


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Transform-0.0+352f0552656f62183b3c2213824430061e98d4ea3a9b5075f5af3274b4849e9e-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Langkah selanjutnya adalah melakukan tuning hyperparameter dengan menggunakan `Tuner` untuk mencari hyperparameter terbaik pada model yang digunakan. Pada kasus ini, akan dilakukan tuning hyperparameter pada beberapa parameter yaitu:
- `num_layers` : dengan nilai range 1 hingga 3
- `hidden_units` : dengan nilai range 32 hingga 512 dengan step 32
- `dropout_rate` : dengan nilai range 0.1 hingga 0.5
- `learning_rate` : dengan nilai [1e-2, 1e-3, 1e-4]

In [12]:
tuner = Tuner(
    module_file=os.path.abspath(os.path.join('modules', 'tuner.py')),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],   
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=8000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=6000),
)

context.run(tuner)

Trial 20 Complete [00h 01m 58s]
val_binary_accuracy: 0.6238216161727905

Best val_binary_accuracy So Far: 0.6301015615463257
Total elapsed time: 00h 38m 08s
Results summary
Results in pipelines/churn-pipeline/.temp/6/churn
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
num_layers: 2
units_0: 64
dropout_0: 0.4
learning_rate: 0.0001
units_1: 224
dropout_1: 0.30000000000000004
units_2: 416
dropout_2: 0.4
units_3: 96
dropout_3: 0.2
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6301015615463257

Trial 19 summary
Hyperparameters:
num_layers: 2
units_0: 160
dropout_0: 0.2
learning_rate: 0.0001
units_1: 224
dropout_1: 0.30000000000000004
units_2: 192
dropout_2: 0.2
units_3: 384
dropout_3: 0.2
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6238216161727905

Trial 06 summary
Hyperparameters:
num_layers: 2
units_0: 320
dropout_0: 0.30000000000000004
learning_rate: 0

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Setelah mendapatkan hyperparameter terbaik, selanjutnya akan dilakukan training model dengan menggunakan `Trainer` dengan hyperparameter yang sudah didapatkan sebelumnya.

In [13]:
trainer = Trainer(
    module_file=os.path.abspath(os.path.join('modules', 'trainer.py')),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=8000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=6000)
)

context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
copying tuner.py -> build/lib


/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmpdnn83fth
running install
running install_lib
copying build/lib/tuner.py -> /tmp/tmpdnn83fth
copying build/lib/trainer.py -> /tmp/tmpdnn83fth
copying build/lib/transform.py -> /tmp/tmpdnn83fth
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpdnn83fth/tfx_user_code_Trainer-0.0+352f0552656f62183b3c2213824430061e98d4ea3a9b5075f5af3274b4849e9e-py3.10.egg-info
running install_scripts
creating /tmp/tmpdnn83fth/tfx_user_code_Trainer-0.0+352f0552656f62183b3c2213824430061e98d4ea3a9b5075f5af3274b4849

Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Trainer-0.0+352f0552656f62183b3c2213824430061e98d4ea3a9b5075f5af3274b4849e9e-py3-none-any.whl



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Dengan `Resolver` akan memilih model terbaru yang akan digunakan untuk melakukan prediksi dengan memberikan tag `BLESSED` jika memenuhi kriteria evaluasi yang ditentukan.

In [14]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menentukan metrik evaluasi yang digunakan untuk mengevaluasi model yang sudah di training. Pada kasus ini, akan menggunakan metrik `accuracy`, `AUC`, serta `confusion matrix`. Pada kasus ini threshold yang digunakan adalah 0.5, untuk menentukan apakah model tersebut dapat memprediksi dengan baik atau tidak. 

In [15]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Churn')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

Dengan `Evaluator` akan melakukan evaluasi model yang sudah di training sebelumnya dengan menggunakan metrik yang sudah ditentukan sebelumnya.

In [16]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Lalu akan dilakukan visualisasi dari hasil evaluasi model yang sudah di training sebelumnya.

In [18]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

Dari hasil evaluasi di atas, dapat diketahui bahwa model yang sudah di training memiliki akurasi sebesar 0.619 dan AUC sebesar 0.669. Hal ini menunjukkan model dapat digunakan untuk memprediksi berdasarkan threshold yang sudah ditentukan sebelumnya.

Dengan `Pusher` akan melakukan push model yang sudah di training sebelumnya ke dalam storage, sehingga model tersebut dapat digunakan untuk melakukan prediksi ataupun deployment model.

In [19]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/customer-churn-model'
        )
    )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [20]:
context.show(pusher.outputs['pushed_model'])

Selanjutnya akan dilakukan pembuatan docker image dari model yang sudah di training sebelumnya, berdasarkan Dockerfile yang sudah disediakan.

In [21]:
!docker build -t mausneg/customer-churn-tf-serving .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.io/tensorflow/serving:latest       0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.io/tensorflow/serving:latest       0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.

Selanjutnya docker image yang sudah dibuat akan di run pada local machine, sehingga model dapat digunakan untuk melakukan prediksi.

In [22]:
!docker run -p 8080:8501 -d mausneg/customer-churn-tf-serving

36aa7165dc667c15a886840be69076f16ded223b54f55a1e41d58343c45842cd


Sebelum melakukan prediksi, akan dilakukan pengecekan apakah model sudah berjalan dengan baik atau tidak dengan melakukan request ke dalam model yang sudah di deploy.

In [24]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/customer-churn-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1730900984'}]}


Dari hasil di atas, dapat diketahui bahwa model sudah berjalan dengan baik dan dapat digunakan untuk melakukan prediksi.